<a href="https://colab.research.google.com/github/PPancham/PhD/blob/main/Multilayer_Perceptrons.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import datetime
import joblib
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
# Removed seaborn to avoid circular dependency issues
from google.colab import files  # For file handling in Colab

def OutlierRemoval(data, columns, filter_data=True):
    """
    Remove outliers from specified columns using IQR method

    Parameters:
    -----------
    data : pandas DataFrame
        The data containing the columns to process
    columns : list
        List of column names to process for outlier removal
    filter_data : bool
        If True, remove outliers; if False, just print message

    Returns:
    --------
    pandas DataFrame
        Data with outliers removed (if filter_data=True)
    """
    if filter_data == False:
        print("\n--- Data has outliers and was not filtered ---")
        return data
    else:
        data_filtered = data.copy()
        removed_count = 0

        for column in columns:
            if column not in data.columns:
                continue

            # Handle non-numeric columns
            if data_filtered[column].dtype == 'object':
                print(f"Skipping outlier removal for non-numeric column: {column}")
                continue

            # Handle columns with NaN values
            if data_filtered[column].isna().any():
                print(f"Column {column} has {data_filtered[column].isna().sum()} NaN values")
                # Fill NaN with median for outlier calculation
                data_filtered[column] = data_filtered[column].fillna(data_filtered[column].median())

            Q1 = data_filtered[column].quantile(0.25)
            Q3 = data_filtered[column].quantile(0.75)
            IQR = Q3 - Q1

            lower_bound = Q1 - 1.5 * IQR
            upper_bound = Q3 + 1.5 * IQR

            outliers = data_filtered[(data_filtered[column] < lower_bound) |
                                    (data_filtered[column] > upper_bound)]
            removed_count += len(outliers)

            data_filtered = data_filtered[(data_filtered[column] >= lower_bound) &
                                        (data_filtered[column] <= upper_bound)]

        print(f"Removed {removed_count} outliers from {len(data) - len(data_filtered)} rows")
        return data_filtered

def SaveModel(model, prefix="Trained_Model"):
    """
    Save the trained model to disk

    Parameters:
    -----------
    model : scikit-learn model
        The trained model to save
    prefix : str
        Prefix for the filename

    Returns:
    --------
    str
        Message confirming model was saved
    """
    filename = f"{prefix}_{datetime.date.today()}.pkl"
    joblib.dump(model, filename)
    return f"Model saved to '{filename}'"

def preprocess_data(data, feature_columns):
    """
    Preprocess data by handling missing values and converting types

    Parameters:
    -----------
    data : pandas DataFrame
        The data to preprocess
    feature_columns : list
        List of column names to use as features

    Returns:
    --------
    pandas DataFrame
        Preprocessed data
    """
    # Make a copy to avoid modifying the original
    processed_data = data.copy()

    # Process only the columns we need
    for col in feature_columns:
        if col not in processed_data.columns:
            continue

        # Convert to numeric, coercing errors
        processed_data[col] = pd.to_numeric(processed_data[col], errors='coerce')

        # Fill missing values with mean
        if processed_data[col].isna().any():
            mean_val = processed_data[col].mean()
            processed_data[col] = processed_data[col].fillna(mean_val)
            print(f"Filled {processed_data[col].isna().sum()} missing values in {col}")

    return processed_data

# Main script
print("Immunoglobulin Analysis with Multilayer Perceptron Model")
print("-" * 60)

# Define columns to process (immunoglobulin measurements)
columns_to_process = ['IgG1 Average', 'IgG2 Average', 'IgG3 Average',
                      'IgG4 Average', 'IgA Average', 'IgE Average', 'IgM Average']

try:
    # In Google Colab environment
    print("Please upload your Excel file with immunoglobulin data")
    uploadedSpreadsheet = files.upload()
    fileName = list(uploadedSpreadsheet.keys())[0]
    data = pd.read_excel(fileName)
    print(f"Successfully loaded data from {fileName}")
    print(f"Data shape: {data.shape[0]} rows, {data.shape[1]} columns")

    # Check if all required columns exist
    missing_columns = [col for col in columns_to_process if col not in data.columns]
    if missing_columns:
        print(f"Warning: Missing columns: {missing_columns}")
        print(f"Available columns: {data.columns.tolist()}")
    else:
        print("All required immunoglobulin columns found")

    # Check if Group column exists
    if 'Group' not in data.columns:
        print("Error: 'Group' column not found in data")
        raise ValueError("Required 'Group' column missing")

    # Preprocess data - handle missing values and type conversion
    data = preprocess_data(data, columns_to_process)

    # Remove outliers
    filter_data = True
    data = OutlierRemoval(data, [col for col in columns_to_process if col in data.columns], filter_data)

    # Save processed data to Excel file
    output_file = "after_outlier_removal.xlsx"
    data.to_excel(output_file, index=False)
    print(f"Data saved to '{output_file}'")

    # Data preparation (feature scaling is essential for MLPs)
    X = data[[col for col in columns_to_process if col in data.columns]]
    y = data['Group']

    # Convert categorical target to numeric using LabelEncoder
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)
    print(f"Target labels encoded: {dict(zip(label_encoder.classes_, range(len(label_encoder.classes_))))}")

    print(f"Features shape: {X.shape}")
    print(f"Target groups: {np.unique(y)}")

except Exception as e:
    print(f"Error loading data: {e}")
    print("Using placeholder data for demonstration")
    # Placeholder for demonstration
    X = np.random.rand(100, len(columns_to_process))
    y_raw = np.random.choice(['Group1', 'Group2', 'Group3'], size=100)

    # Encode the placeholder data
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y_raw)

    print("Using placeholder data with groups:", label_encoder.classes_)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, test_size=0.3, random_state=23, stratify=y_encoded
)

# Scale features (very important for neural networks)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Create and train Multilayer Perceptron model
print("\nTraining Multilayer Perceptron model...")
mlp = MLPClassifier(
    hidden_layer_sizes=(100, 50),  # Two hidden layers with 100 and 50 neurons
    activation='relu',              # ReLU activation function
    solver='adam',                  # Adam optimizer
    alpha=0.0001,                   # L2 regularization term
    batch_size='auto',              # Batch size for gradient descent
    learning_rate='adaptive',       # Adaptive learning rate
    max_iter=1000,                  # Maximum number of iterations
    random_state=42,                # For reproducibility
    early_stopping=True,            # Use early stopping
    validation_fraction=0.1,        # Fraction of training data for validation
    n_iter_no_change=10             # Stop if no improvement after 10 iterations
)

# Train the model
mlp.fit(X_train_scaled, y_train)

# Make predictions
y_pred = mlp.predict(X_test_scaled)

# Evaluate the model
print(f'Accuracy: {accuracy_score(y_test, y_pred):.4f}')
print('\nClassification Report:')
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))

# Decode predictions back to original labels for better interpretability
y_pred_decoded = label_encoder.inverse_transform(y_pred)
y_test_decoded = label_encoder.inverse_transform(y_test)

# Print results with original group labels
print("\nPrediction Results (Original Labels):")
for true, pred in zip(y_test_decoded[:10], y_pred_decoded[:10]):  # Show first 10 examples
    print(f"True: {true} | Predicted: {pred} | {'✓' if true == pred else '✗'}")

# Confusion Matrix visualization
def plot_confusion_matrix(y_true, y_pred, class_names=None):
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(10, 8))
    plt.imshow(cm, interpolation='nearest', cmap='Blues')
    plt.colorbar()

    # Add labels
    tick_marks = np.arange(len(class_names) if class_names is not None else cm.shape[0])
    plt.xticks(tick_marks, class_names if class_names is not None else tick_marks)
    plt.yticks(tick_marks, class_names if class_names is not None else tick_marks)

    # Add text annotations
    thresh = cm.max() / 2
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            plt.text(j, i, format(cm[i, j], 'd'),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")

    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Confusion Matrix')
    plt.tight_layout()
    plt.savefig('confusion_matrix.png')
    plt.close()

# Plot confusion matrix with original class names
plot_confusion_matrix(y_test, y_pred, class_names=label_encoder.classes_)
print("Confusion matrix saved as 'confusion_matrix.png'")

# Save the model
print("\nSaving model...")
save_msg = SaveModel(mlp, prefix="MLP_Immunoglobulin_Model")
print(save_msg)

print("\nMLP Model training and evaluation complete")

Immunoglobulin Analysis with Multilayer Perceptron Model
------------------------------------------------------------
Please upload your Excel file with immunoglobulin data


Saving Biomarker_16032025_without_SWD_with_GNV.xlsx to Biomarker_16032025_without_SWD_with_GNV (1).xlsx
Successfully loaded data from Biomarker_16032025_without_SWD_with_GNV (1).xlsx
Data shape: 389 rows, 10 columns
All required immunoglobulin columns found
Removed 133 outliers from 133 rows
Data saved to 'after_outlier_removal.xlsx'
Target labels encoded: {'AD': 0, 'Cognitive Negative': 1, 'Cognitive Positive': 2, 'MCI': 3}
Features shape: (256, 7)
Target groups: ['AD' 'Cognitive Negative' 'Cognitive Positive' 'MCI']

Training Multilayer Perceptron model...
Accuracy: 0.5195

Classification Report:
                    precision    recall  f1-score   support

                AD       0.00      0.00      0.00        13
Cognitive Negative       0.50      0.91      0.65        23
Cognitive Positive       0.33      0.25      0.29        20
               MCI       0.70      0.67      0.68        21

          accuracy                           0.52        77
         macro avg       0.38   

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Confusion matrix saved as 'confusion_matrix.png'

Saving model...
Model saved to 'MLP_Immunoglobulin_Model_2025-03-28.pkl'

MLP Model training and evaluation complete


In [ ]:
import pandas as pd
import numpy as np
import datetime
import joblib
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder, label_binarize
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_curve, auc
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt
from google.colab import files  # For file handling in Colab

def OutlierRemoval(data, columns, filter_data=True):
    """
    Remove outliers from specified columns using IQR method

    Parameters:
    -----------
    data : pandas DataFrame
        The data containing the columns to process
    columns : list
        List of column names to process for outlier removal
    filter_data : bool
        If True, remove outliers; if False, just print message

    Returns:
    --------
    pandas DataFrame
        Data with outliers removed (if filter_data=True)
    """
    if filter_data == False:
        print("\n--- Data has outliers and was not filtered ---")
        return data
    else:
        data_filtered = data.copy()
        removed_count = 0

        for column in columns:
            if column not in data.columns:
                continue

            # Handle non-numeric columns
            if data_filtered[column].dtype == 'object':
                print(f"Skipping outlier removal for non-numeric column: {column}")
                continue

            # Handle columns with NaN values
            if data_filtered[column].isna().any():
                print(f"Column {column} has {data_filtered[column].isna().sum()} NaN values")
                # Fill NaN with median for outlier calculation
                data_filtered[column] = data_filtered[column].fillna(data_filtered[column].median())

            Q1 = data_filtered[column].quantile(0.25)
            Q3 = data_filtered[column].quantile(0.75)
            IQR = Q3 - Q1

            lower_bound = Q1 - 1.5 * IQR
            upper_bound = Q3 + 1.5 * IQR

            outliers = data_filtered[(data_filtered[column] < lower_bound) |
                                    (data_filtered[column] > upper_bound)]
            removed_count += len(outliers)

            data_filtered = data_filtered[(data_filtered[column] >= lower_bound) &
                                        (data_filtered[column] <= upper_bound)]

        print(f"Removed {removed_count} outliers from {len(data) - len(data_filtered)} rows")
        return data_filtered

def SaveModel(model, prefix="Trained_Model"):
    """
    Save the trained model to disk

    Parameters:
    -----------
    model : scikit-learn model
        The trained model to save
    prefix : str
        Prefix for the filename

    Returns:
    --------
    str
        Message confirming model was saved
    """
    filename = f"{prefix}_{datetime.date.today()}.pkl"
    joblib.dump(model, filename)
    return f"Model saved to '{filename}'"

def preprocess_data(data, feature_columns):
    """
    Preprocess data by handling missing values and converting types

    Parameters:
    -----------
    data : pandas DataFrame
        The data to preprocess
    feature_columns : list
        List of column names to use as features

    Returns:
    --------
    pandas DataFrame
        Preprocessed data
    """
    # Make a copy to avoid modifying the original
    processed_data = data.copy()

    # Process only the columns we need
    for col in feature_columns:
        if col not in processed_data.columns:
            continue

        # Convert to numeric, coercing errors
        processed_data[col] = pd.to_numeric(processed_data[col], errors='coerce')

        # Fill missing values with mean
        if processed_data[col].isna().any():
            mean_val = processed_data[col].mean()
            processed_data[col] = processed_data[col].fillna(mean_val)
            print(f"Filled {processed_data[col].isna().sum()} missing values in {col}")

    return processed_data

def engineer_features(data, base_columns):
    """
    Create engineered features from the base immunoglobulin measurements

    Parameters:
    -----------
    data : pandas DataFrame
        The data containing the base columns
    base_columns : list
        List of base column names

    Returns:
    --------
    pandas DataFrame
        Data with additional engineered features
    """
    # Make a copy to avoid modifying the original
    df = data.copy()

    # Create ratios between different immunoglobulins
    for i, col1 in enumerate(base_columns):
        for col2 in base_columns[i+1:]:
            if col1 in df.columns and col2 in df.columns:
                ratio_name = f"{col1}_to_{col2}"
                # Avoid division by zero
                df[ratio_name] = df[col1] / (df[col2] + 1e-10)
                print(f"Created ratio feature: {ratio_name}")

    # Create sum and average features
    valid_columns = [col for col in base_columns if col in df.columns]
    if len(valid_columns) > 1:
        df['total_ig'] = df[valid_columns].sum(axis=1)
        print("Created feature: total_ig (sum of all immunoglobulins)")

        # Create proportions (percentage of each Ig to the total)
        for col in valid_columns:
            prop_name = f"{col}_proportion"
            df[prop_name] = df[col] / (df['total_ig'] + 1e-10)
            print(f"Created proportion feature: {prop_name}")

    return df

def plot_confusion_matrix(y_true, y_pred, class_names=None):
    """
    Plot confusion matrix

    Parameters:
    -----------
    y_true : array-like
        True labels
    y_pred : array-like
        Predicted labels
    class_names : list, optional
        List of class names
    """
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(10, 8))
    plt.imshow(cm, interpolation='nearest', cmap='Blues')
    plt.colorbar()

    # Add labels
    tick_marks = np.arange(len(class_names) if class_names is not None else cm.shape[0])
    plt.xticks(tick_marks, class_names if class_names is not None else tick_marks)
    plt.yticks(tick_marks, class_names if class_names is not None else tick_marks)

    # Add text annotations
    thresh = cm.max() / 2
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            plt.text(j, i, format(cm[i, j], 'd'),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")

    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Confusion Matrix')
    plt.tight_layout()
    plt.savefig('confusion_matrix.png')
    print("Saved confusion matrix to 'confusion_matrix.png'")
    plt.close()

def plot_roc_curves(y_test, y_pred_proba, class_names):
    """
    Plot ROC curves for each class (one-vs-rest)

    Parameters:
    -----------
    y_test : array-like
        True labels
    y_pred_proba : array-like
        Predicted probabilities for each class
    class_names : list
        List of class names
    """
    # Convert y_test to one-hot encoding for ROC curve calculation
    y_test_bin = label_binarize(y_test, classes=range(len(class_names)))

    plt.figure(figsize=(10, 8))

    # Calculate ROC curve and ROC area for each class
    for i, class_name in enumerate(class_names):
        fpr, tpr, _ = roc_curve(y_test_bin[:, i], y_pred_proba[:, i])
        roc_auc = auc(fpr, tpr)
        plt.plot(fpr, tpr, lw=2,
                 label=f'ROC curve for {class_name} (AUC = {roc_auc:.2f})')

    # Plot diagonal line
    plt.plot([0, 1], [0, 1], 'k--', lw=2)
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curves for Each Class')
    plt.legend(loc="lower right")
    plt.tight_layout()
    plt.savefig('roc_curves.png')
    print("Saved ROC curves to 'roc_curves.png'")
    plt.close()

def run_random_forest_model(X_train_resampled, y_train_resampled, X_test_scaled, y_test, feature_columns, label_encoder):
    """
    Train and evaluate a RandomForest model

    Parameters:
    -----------
    X_train_resampled : array-like
        Resampled training features
    y_train_resampled : array-like
        Resampled training labels
    X_test_scaled : array-like
        Scaled test features
    y_test : array-like
        Test labels
    feature_columns : list
        Feature column names for importance plots
    label_encoder : LabelEncoder
        Encoder to convert numeric labels back to strings

    Returns:
    --------
    tuple
        (trained model, accuracy)
    """
    print("\nTraining RandomForest model...")
    rf = RandomForestClassifier(
        n_estimators=200,              # More trees
        max_depth=None,                # Allow deep trees
        min_samples_split=2,           # Default
        min_samples_leaf=1,            # Default
        max_features='sqrt',           # Common setting for classification
        bootstrap=True,                # Use bootstrap samples
        class_weight='balanced',       # Handle class imbalance
        random_state=42,               # For reproducibility
        verbose=1                      # Show progress
    )

    # Train the model
    rf.fit(X_train_resampled, y_train_resampled)

    # Make predictions
    y_pred = rf.predict(X_test_scaled)

    # Get prediction probabilities for ROC curve
    y_pred_proba = rf.predict_proba(X_test_scaled)

    # Evaluate the model
    print("\nRandomForest Model Evaluation:")
    acc = accuracy_score(y_test, y_pred)
    print(f'Accuracy: {acc:.4f}')

    print('\nClassification Report:')
    report = classification_report(y_test, y_pred,
                                  target_names=label_encoder.classes_,
                                  zero_division=0)
    print(report)

    # Plot confusion matrix
    plot_confusion_matrix(y_test, y_pred, class_names=label_encoder.classes_)

    # Plot ROC curves
    try:
        plot_roc_curves(y_test, y_pred_proba, label_encoder.classes_)
    except Exception as e:
        print(f"Could not generate ROC curves: {e}")

    # Get feature importances
    feature_importances = rf.feature_importances_

    # Plot feature importance
    plt.figure(figsize=(12, 8))
    indices = np.argsort(feature_importances)[::-1]
    top_indices = indices[:15]  # Show top 15 features
    plt.barh(range(len(top_indices)), feature_importances[top_indices], align='center')
    plt.yticks(range(len(top_indices)), np.array(feature_columns)[top_indices])
    plt.tight_layout()
    plt.savefig('rf_feature_importance.png')
    plt.close()
    print("Feature importance plot saved as 'rf_feature_importance.png'")

    return rf, acc

# Main script starts here
print("Complete Immunoglobulin Classification Model")
print("=" * 60)

# Define base columns to process (immunoglobulin measurements)
base_columns = ['IgG1 Average', 'IgG2 Average', 'IgG3 Average',
                'IgG4 Average', 'IgA Average', 'IgE Average', 'IgM Average']

try:
    # In Google Colab environment
    print("Please upload your Excel file with immunoglobulin data")
    uploadedSpreadsheet = files.upload()
    fileName = list(uploadedSpreadsheet.keys())[0]
    data = pd.read_excel(fileName)
    print(f"Successfully loaded data from {fileName}")
    print(f"Data shape: {data.shape[0]} rows, {data.shape[1]} columns")

    # Check if all required columns exist
    missing_columns = [col for col in base_columns if col not in data.columns]
    if missing_columns:
        print(f"Warning: Missing columns: {missing_columns}")
        print(f"Available columns: {data.columns.tolist()}")
    else:
        print("All required immunoglobulin columns found")

    # Check if Group column exists
    if 'Group' not in data.columns:
        print("Warning: 'Group' column not found in data")
        group_col = None
        # Look for alternative column names
        possible_group_cols = ['group', 'GROUP', 'Category', 'class', 'Class', 'type', 'Type']
        for col in possible_group_cols:
            if col in data.columns:
                print(f"Using '{col}' as the target column")
                group_col = col
                break

        if group_col is None:
            # Ask user to specify which column to use
            print("Available columns:")
            for i, col in enumerate(data.columns):
                print(f"{i}: {col}")
            # In a real script, you would need to get user input here
            # For now, assume we use a specific column
            group_col = 'Group'
            print(f"Using default 'Group' column name")
    else:
        group_col = 'Group'

    # Preprocess data - handle missing values and type conversion
    data = preprocess_data(data, base_columns)

    # Remove outliers
    filter_data = True
    data = OutlierRemoval(data, [col for col in base_columns if col in data.columns], filter_data)

    # Engineer additional features
    data = engineer_features(data, base_columns)

    # Save processed data to Excel file
    output_file = "processed_immunoglobulin_data.xlsx"
    data.to_excel(output_file, index=False)
    print(f"Processed data saved to '{output_file}'")

    # Get all feature columns (original + engineered)
    feature_columns = [col for col in data.columns
                      if col != group_col and col != 'Sample ID' and col != 'Cohort']
    print(f"Total features: {len(feature_columns)}")

    # Data preparation
    X = data[feature_columns]
    y = data[group_col]

    # Convert categorical target to numeric using LabelEncoder
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)
    print(f"Target labels encoded: {dict(zip(label_encoder.classes_, range(len(label_encoder.classes_))))}")

    # Split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(
        X, y_encoded, test_size=0.25, random_state=42, stratify=y_encoded
    )

    # Scale features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Apply SMOTE for class balancing
    print("\nApplying SMOTE for class balancing...")
    smote = SMOTE(random_state=42)
    X_train_resampled, y_train_resampled = smote.fit_resample(X_train_scaled, y_train)

    # Class distribution before and after SMOTE
    classes, counts = np.unique(y_train, return_counts=True)
    print("Original training class distribution:")
    for cls, count in zip(classes, counts):
        class_name = label_encoder.inverse_transform([cls])[0]
        print(f"  {class_name}: {count}")

    classes, counts = np.unique(y_train_resampled, return_counts=True)
    print("After SMOTE class distribution:")
    for cls, count in zip(classes, counts):
        class_name = label_encoder.inverse_transform([cls])[0]
        print(f"  {class_name}: {count}")

    # Try both models and compare
    models = {}

    # 1. MLP Model
    print("\n" + "="*40)
    print("TRAINING MLP MODEL")
    print("="*40)

    # Create and train optimized MLP model
    print("\nTraining optimized MLP model...")
    mlp = MLPClassifier(
        hidden_layer_sizes=(200, 100, 50),  # Three hidden layers
        activation='tanh',                  # Try tanh instead of relu
        solver='adam',                      # Adam optimizer
        alpha=0.001,                        # Increased regularization
        batch_size='auto',                  # Auto batch size
        learning_rate='adaptive',           # Adaptive learning rate
        max_iter=2000,                      # Increased max iterations
        random_state=42,                    # For reproducibility
        early_stopping=True,                # Use early stopping
        validation_fraction=0.1,            # Fraction for validation
        n_iter_no_change=20,                # More patience
        # Note: MLPClassifier doesn't support class_weight parameter
        # We're using SMOTE instead for class balancing
        verbose=True                        # Show training progress
    )

    # Train the model
    mlp.fit(X_train_resampled, y_train_resampled)

    # Make predictions
    y_pred = mlp.predict(X_test_scaled)

    # Get prediction probabilities for ROC curve
    y_pred_proba = mlp.predict_proba(X_test_scaled)

    # Evaluate the model
    print("\nMLP Model Evaluation:")
    acc = accuracy_score(y_test, y_pred)
    print(f'Accuracy: {acc:.4f}')

    print('\nClassification Report:')
    report = classification_report(y_test, y_pred,
                                  target_names=label_encoder.classes_,
                                  zero_division=0)
    print(report)

    # Plot confusion matrix
    plot_confusion_matrix(y_test, y_pred, class_names=label_encoder.classes_)

    # Plot ROC curves
    try:
        plot_roc_curves(y_test, y_pred_proba, label_encoder.classes_)
    except Exception as e:
        print(f"Could not generate ROC curves: {e}")

    # Save the MLP model
    models['mlp'] = (mlp, acc)
    mlp_model_filename = "MLP_Immunoglobulin_Model.pkl"
    joblib.dump(mlp, mlp_model_filename)
    print(f"\nMLP model saved to '{mlp_model_filename}'")

    # 2. Random Forest Model
    print("\n" + "="*40)
    print("TRAINING RANDOM FOREST MODEL")
    print("="*40)

    rf_model, rf_acc = run_random_forest_model(
        X_train_resampled, y_train_resampled, X_test_scaled, y_test,
        feature_columns, label_encoder
    )

    models['rf'] = (rf_model, rf_acc)
    rf_model_filename = "RF_Immunoglobulin_Model.pkl"
    joblib.dump(rf_model, rf_model_filename)
    print(f"\nRandomForest model saved to '{rf_model_filename}'")

    # Compare model performances
    print("\n" + "="*40)
    print("MODEL COMPARISON")
    print("="*40)
    for name, (model, acc) in models.items():
        print(f"{name.upper()} accuracy: {acc:.4f}")

    # Determine best model
    best_model_name = max(models, key=lambda k: models[k][1])
    best_model, best_acc = models[best_model_name]

    print(f"\nBest model: {best_model_name.upper()} with accuracy {best_acc:.4f}")

    # Save the preprocessing components for later use
    preprocessing = {
        'scaler': scaler,
        'label_encoder': label_encoder,
        'feature_columns': feature_columns
    }
    preprocessing_filename = "immunoglobulin_preprocessing.pkl"
    joblib.dump(preprocessing, preprocessing_filename)
    print(f"Preprocessing components saved to '{preprocessing_filename}'")

    # Cross-validation for more robust evaluation of best model
    print("\nPerforming 5-fold cross-validation on best model...")
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    if best_model_name == 'mlp':
        # Pipeline for MLP
        pipeline = Pipeline([
            ('scaler', StandardScaler()),
            ('classifier', MLPClassifier(
                hidden_layer_sizes=(200, 100, 50),
                activation='tanh',
                solver='adam',
                alpha=0.001,
                max_iter=2000,
                random_state=42,
                early_stopping=True,
                # No class_weight parameter
            ))
        ])
    else:  # RandomForest
        # Pipeline for RF
        pipeline = Pipeline([
            ('scaler', StandardScaler()),
            ('classifier', RandomForestClassifier(
                n_estimators=200,
                max_features='sqrt',
                class_weight='balanced',
                random_state=42
            ))
        ])

    # Get cross-validation scores
    cv_scores = cross_val_score(pipeline, X, y_encoded, cv=cv, scoring='accuracy')
    print(f"Cross-validation accuracy scores: {cv_scores}")
    print(f"Mean CV accuracy: {np.mean(cv_scores):.4f} ± {np.std(cv_scores):.4f}")

except Exception as e:
    print(f"Error: {e}")
    import traceback
    traceback.print_exc()

print("\nImmunoglobulin classification analysis complete!")

Complete Immunoglobulin Classification Model
Please upload your Excel file with immunoglobulin data


Saving Biomarker_16032025_without_SWD_with_GNV.xlsx to Biomarker_16032025_without_SWD_with_GNV (4).xlsx
Successfully loaded data from Biomarker_16032025_without_SWD_with_GNV (4).xlsx
Data shape: 389 rows, 10 columns
All required immunoglobulin columns found
Removed 133 outliers from 133 rows
Created ratio feature: IgG1 Average_to_IgG2 Average
Created ratio feature: IgG1 Average_to_IgG3 Average
Created ratio feature: IgG1 Average_to_IgG4 Average
Created ratio feature: IgG1 Average_to_IgA Average
Created ratio feature: IgG1 Average_to_IgE Average
Created ratio feature: IgG1 Average_to_IgM Average
Created ratio feature: IgG2 Average_to_IgG3 Average
Created ratio feature: IgG2 Average_to_IgG4 Average
Created ratio feature: IgG2 Average_to_IgA Average
Created ratio feature: IgG2 Average_to_IgE Average
Created ratio feature: IgG2 Average_to_IgM Average
Created ratio feature: IgG3 Average_to_IgG4 Average
Created ratio feature: IgG3 Average_to_IgA Average
Created ratio feature: IgG3 Average_to

[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.2s
[Parallel(n_jobs=1)]: Done 199 tasks      | elapsed:    1.3s
[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 199 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 199 tasks      | elapsed:    0.0s



RandomForest Model Evaluation:
Accuracy: 0.8281

Classification Report:
                    precision    recall  f1-score   support

                AD       0.86      0.55      0.67        11
Cognitive Negative       0.84      0.84      0.84        19
Cognitive Positive       0.88      0.88      0.88        17
               MCI       0.76      0.94      0.84        17

          accuracy                           0.83        64
         macro avg       0.84      0.80      0.81        64
      weighted avg       0.83      0.83      0.82        64

Saved confusion matrix to 'confusion_matrix.png'
Saved ROC curves to 'roc_curves.png'
Feature importance plot saved as 'rf_feature_importance.png'

RandomForest model saved to 'RF_Immunoglobulin_Model.pkl'

MODEL COMPARISON
MLP accuracy: 0.7031
RF accuracy: 0.8281

Best model: RF with accuracy 0.8281
Preprocessing components saved to 'immunoglobulin_preprocessing.pkl'

Performing 5-fold cross-validation on best model...
Cross-validation acc

In [ ]:
import pandas as pd
import numpy as np
import datetime
import joblib
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_curve, auc
from imblearn.over_sampling import SMOTE
from google.colab import files  # For file handling in Colab

def OutlierRemoval(data, columns, filter_data=True):
    """
    Remove outliers from specified columns using IQR method
    """
    if filter_data == False:
        print("\n--- Data has outliers and was not filtered ---")
        return data
    else:
        data_filtered = data.copy()
        removed_count = 0

        for column in columns:
            if column not in data.columns:
                continue

            # Handle non-numeric columns
            if data_filtered[column].dtype == 'object':
                print(f"Skipping outlier removal for non-numeric column: {column}")
                continue

            # Handle columns with NaN values
            if data_filtered[column].isna().any():
                print(f"Column {column} has {data_filtered[column].isna().sum()} NaN values")
                data_filtered[column] = data_filtered[column].fillna(data_filtered[column].median())

            Q1 = data_filtered[column].quantile(0.25)
            Q3 = data_filtered[column].quantile(0.75)
            IQR = Q3 - Q1

            lower_bound = Q1 - 1.5 * IQR
            upper_bound = Q3 + 1.5 * IQR

            outliers = data_filtered[(data_filtered[column] < lower_bound) |
                                    (data_filtered[column] > upper_bound)]
            removed_count += len(outliers)

            data_filtered = data_filtered[(data_filtered[column] >= lower_bound) &
                                        (data_filtered[column] <= upper_bound)]

        print(f"Removed {removed_count} outliers from {len(data) - len(data_filtered)} rows")
        return data_filtered

def preprocess_data(data, feature_columns):
    """
    Preprocess data by handling missing values and converting types
    """
    processed_data = data.copy()

    for col in feature_columns:
        if col not in processed_data.columns:
            continue

        # Convert to numeric, coercing errors
        processed_data[col] = pd.to_numeric(processed_data[col], errors='coerce')

        # Fill missing values with mean
        if processed_data[col].isna().any():
            mean_val = processed_data[col].mean()
            processed_data[col] = processed_data[col].fillna(mean_val)
            print(f"Filled {processed_data[col].isna().sum()} missing values in {col}")

    return processed_data

def engineer_features(data, base_columns):
    """
    Create engineered features from the base immunoglobulin measurements
    """
    df = data.copy()

    # Create ratios between different immunoglobulins
    for i, col1 in enumerate(base_columns):
        for col2 in base_columns[i+1:]:
            if col1 in df.columns and col2 in df.columns:
                ratio_name = f"{col1}_to_{col2}"
                # Avoid division by zero
                df[ratio_name] = df[col1] / (df[col2] + 1e-10)
                print(f"Created ratio feature: {ratio_name}")

    # Create sum and average features
    valid_columns = [col for col in base_columns if col in df.columns]
    if len(valid_columns) > 1:
        df['total_ig'] = df[valid_columns].sum(axis=1)
        print("Created feature: total_ig (sum of all immunoglobulins)")

        # Create proportions (percentage of each Ig to the total)
        for col in valid_columns:
            prop_name = f"{col}_proportion"
            df[prop_name] = df[col] / (df['total_ig'] + 1e-10)
            print(f"Created proportion feature: {prop_name}")

    return df

def plot_confusion_matrix(y_true, y_pred, class_names=None):
    """
    Plot confusion matrix
    """
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(10, 8))
    plt.imshow(cm, interpolation='nearest', cmap='Blues')
    plt.colorbar()

    # Add labels
    tick_marks = np.arange(len(class_names) if class_names is not None else cm.shape[0])
    plt.xticks(tick_marks, class_names if class_names is not None else tick_marks)
    plt.yticks(tick_marks, class_names if class_names is not None else tick_marks)

    # Add text annotations
    thresh = cm.max() / 2
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            plt.text(j, i, format(cm[i, j], 'd'),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")

    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Confusion Matrix')
    plt.tight_layout()
    plt.savefig('rf_confusion_matrix.png')
    print("Saved confusion matrix to 'rf_confusion_matrix.png'")
    plt.close()

def plot_roc_curves(y_test, y_pred_proba, class_names):
    """
    Plot ROC curves for each class (one-vs-rest)
    """
    from sklearn.preprocessing import label_binarize

    # Convert y_test to one-hot encoding for ROC curve calculation
    y_test_bin = label_binarize(y_test, classes=range(len(class_names)))

    plt.figure(figsize=(10, 8))

    # Calculate ROC curve and ROC area for each class
    for i, class_name in enumerate(class_names):
        fpr, tpr, _ = roc_curve(y_test_bin[:, i], y_pred_proba[:, i])
        roc_auc = auc(fpr, tpr)
        plt.plot(fpr, tpr, lw=2,
                 label=f'ROC curve for {class_name} (AUC = {roc_auc:.2f})')

    # Plot diagonal line
    plt.plot([0, 1], [0, 1], 'k--', lw=2)
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curves for Each Class')
    plt.legend(loc="lower right")
    plt.tight_layout()
    plt.savefig('rf_roc_curves.png')
    print("Saved ROC curves to 'rf_roc_curves.png'")
    plt.close()

def plot_feature_importance(rf_model, feature_columns):
    """
    Plot feature importance from RandomForest model
    """
    feature_importances = rf_model.feature_importances_

    # Sort features by importance
    indices = np.argsort(feature_importances)[::-1]

    # Plot top 15 features
    top_indices = indices[:15]
    plt.figure(figsize=(12, 8))
    plt.barh(range(len(top_indices)), feature_importances[top_indices], align='center')
    plt.yticks(range(len(top_indices)), np.array(feature_columns)[top_indices])
    plt.xlabel('Feature Importance')
    plt.title('Top 15 Important Features')
    plt.tight_layout()
    plt.savefig('rf_feature_importance.png')
    print("Feature importance plot saved as 'rf_feature_importance.png'")
    plt.close()

# Main script
print("RandomForest Immunoglobulin Classification Model")
print("=" * 60)

# Define base columns to process (immunoglobulin measurements)
base_columns = ['IgG1 Average', 'IgG2 Average', 'IgG3 Average',
                'IgG4 Average', 'IgA Average', 'IgE Average', 'IgM Average']

try:
    # In Google Colab environment
    print("Please upload your Excel file with immunoglobulin data")
    uploaded_spreadsheet = files.upload()
    file_name = list(uploaded_spreadsheet.keys())[0]
    data = pd.read_excel(file_name)
    print(f"Successfully loaded data from {file_name}")
    print(f"Data shape: {data.shape[0]} rows, {data.shape[1]} columns")

    # Check if all required columns exist
    missing_columns = [col for col in base_columns if col not in data.columns]
    if missing_columns:
        print(f"Warning: Missing columns: {missing_columns}")
        print(f"Available columns: {data.columns.tolist()}")
    else:
        print("All required immunoglobulin columns found")

    # Check if Group column exists
    group_col = 'Group'
    if group_col not in data.columns:
        print(f"Warning: '{group_col}' column not found in data")
        # Look for alternative column names or handle manually

    # Preprocess data - handle missing values and type conversion
    data = preprocess_data(data, base_columns)

    # Remove outliers
    data = OutlierRemoval(data, [col for col in base_columns if col in data.columns], filter_data=False)

    # Engineer additional features
    data = engineer_features(data, base_columns)

    # Save processed data to Excel file
    output_file = "rf_processed_immunoglobulin_data.xlsx"
    data.to_excel(output_file, index=False)
    print(f"Processed data saved to '{output_file}'")

    # Get all feature columns (original + engineered)
    feature_columns = [col for col in data.columns
                      if col != group_col and col != 'Sample ID' and col != 'Cohort']
    print(f"Total features: {len(feature_columns)}")

    # Data preparation
    X = data[feature_columns]
    y = data[group_col]

    # Convert categorical target to numeric using LabelEncoder
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)
    print(f"Target labels encoded: {dict(zip(label_encoder.classes_, range(len(label_encoder.classes_))))}")

    # Split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(
        X, y_encoded, test_size=0.25, random_state=42, stratify=y_encoded
    )

    # Scale features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Apply SMOTE for class balancing
    print("\nApplying SMOTE for class balancing...")
    smote = SMOTE(random_state=42)
    X_train_resampled, y_train_resampled = smote.fit_resample(X_train_scaled, y_train)

    # Class distribution before and after SMOTE
    classes, counts = np.unique(y_train, return_counts=True)
    print("Original training class distribution:")
    for cls, count in zip(classes, counts):
        class_name = label_encoder.inverse_transform([cls])[0]
        print(f"  {class_name}: {count}")

    classes, counts = np.unique(y_train_resampled, return_counts=True)
    print("After SMOTE class distribution:")
    for cls, count in zip(classes, counts):
        class_name = label_encoder.inverse_transform([cls])[0]
        print(f"  {class_name}: {count}")

    # Create and train the RandomForest model
    print("\nTraining RandomForest model...")
    rf = RandomForestClassifier(
        n_estimators=100,              # More trees for better performance
        max_depth=None,                # Allow deep trees
        min_samples_split=2,           # Default value
        min_samples_leaf=1,            # Default value
        max_features='sqrt',           # Common setting for classification
        bootstrap=True,                # Use bootstrap samples
        class_weight='balanced',       # Handle class imbalance
        random_state=42,               # For reproducibility
        verbose=1                      # Show progress
    )

    # Train the model
    rf.fit(X_train_resampled, y_train_resampled)

    # Make predictions
    y_pred = rf.predict(X_test_scaled)

    # Get prediction probabilities for ROC curve
    y_pred_proba = rf.predict_proba(X_test_scaled)

    # Evaluate the model
    print("\nRandomForest Model Evaluation:")
    acc = accuracy_score(y_test, y_pred)
    print(f'Accuracy: {acc:.4f}')

    print('\nClassification Report:')
    report = classification_report(y_test, y_pred,
                                  target_names=label_encoder.classes_,
                                  zero_division=0)
    print(report)

    # Plot confusion matrix
    plot_confusion_matrix(y_test, y_pred, class_names=label_encoder.classes_)

    # Plot ROC curves
    try:
        plot_roc_curves(y_test, y_pred_proba, label_encoder.classes_)
    except Exception as e:
        print(f"Could not generate ROC curves: {e}")

    # Plot feature importance
    plot_feature_importance(rf, feature_columns)

    # Cross-validation for more robust evaluation
    print("\nPerforming 5-fold cross-validation...")
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    # Create a pipeline for cross-validation
    from sklearn.pipeline import Pipeline
    pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('classifier', RandomForestClassifier(
            n_estimators=200,
            max_features='sqrt',
            class_weight='balanced',
            random_state=42
        ))
    ])

    # Get cross-validation scores
    cv_scores = cross_val_score(pipeline, X, y_encoded, cv=cv, scoring='accuracy')
    print(f"Cross-validation accuracy scores: {cv_scores}")
    print(f"Mean CV accuracy: {np.mean(cv_scores):.4f} ± {np.std(cv_scores):.4f}")

    # Save the model
    model_filename = "RF_Immunoglobulin_Model.pkl"
    joblib.dump(rf, model_filename)
    print(f"\nModel saved to '{model_filename}'")

    # Save preprocessing components
    preprocessing = {
        'scaler': scaler,
        'label_encoder': label_encoder,
        'feature_columns': feature_columns
    }
    joblib.dump(preprocessing, "rf_preprocessing_components.pkl")
    print("Preprocessing components saved for future use")

except Exception as e:
    print(f"Error: {e}")
    import traceback
    traceback.print_exc()

print("\nRandomForest classification complete!")

RandomForest Immunoglobulin Classification Model
Please upload your Excel file with immunoglobulin data


Saving Biomarker_16032025_without_SWD_with_GNV.xlsx to Biomarker_16032025_without_SWD_with_GNV.xlsx
Successfully loaded data from Biomarker_16032025_without_SWD_with_GNV.xlsx
Data shape: 389 rows, 10 columns
All required immunoglobulin columns found

--- Data has outliers and was not filtered ---
Created ratio feature: IgG1 Average_to_IgG2 Average
Created ratio feature: IgG1 Average_to_IgG3 Average
Created ratio feature: IgG1 Average_to_IgG4 Average
Created ratio feature: IgG1 Average_to_IgA Average
Created ratio feature: IgG1 Average_to_IgE Average
Created ratio feature: IgG1 Average_to_IgM Average
Created ratio feature: IgG2 Average_to_IgG3 Average
Created ratio feature: IgG2 Average_to_IgG4 Average
Created ratio feature: IgG2 Average_to_IgA Average
Created ratio feature: IgG2 Average_to_IgE Average
Created ratio feature: IgG2 Average_to_IgM Average
Created ratio feature: IgG3 Average_to_IgG4 Average
Created ratio feature: IgG3 Average_to_IgA Average
Created ratio feature: IgG3 Avera

[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.8s
[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.0s



RandomForest Model Evaluation:
Accuracy: 0.8163

Classification Report:
                    precision    recall  f1-score   support

                AD       0.59      0.77      0.67        13
Cognitive Negative       0.84      0.76      0.80        34
Cognitive Positive       0.90      0.90      0.90        29
               MCI       0.86      0.82      0.84        22

          accuracy                           0.82        98
         macro avg       0.80      0.81      0.80        98
      weighted avg       0.83      0.82      0.82        98

Saved confusion matrix to 'rf_confusion_matrix.png'
Saved ROC curves to 'rf_roc_curves.png'
Feature importance plot saved as 'rf_feature_importance.png'

Performing 5-fold cross-validation...
Cross-validation accuracy scores: [0.82051282 0.83333333 0.79487179 0.84615385 0.80519481]
Mean CV accuracy: 0.8200 ± 0.0185

Model saved to 'RF_Immunoglobulin_Model.pkl'
Preprocessing components saved for future use

RandomForest classification comple